Modelo Transfer learning para proyecto de clasificacion de insecto (preguntar al profe sobre porque no funciona el modelo que se utiliza en este notebook en un entorno local)

In [ ]:
import zipfile
import os
zip_ref = zipfile.ZipFile('insects.v2-release.multiclass.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()
os.listdir('/content')

['.config',
 'insects.v2-release.multiclass.zip',
 'README.roboflow.txt',
 'mi_modelo.h5',
 'mi_modelo_savedmodel',
 'train',
 'README.dataset.txt',
 'test',
 'valid',
 'sample_data']

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_df = pd.read_csv('/content/train/_classes.csv')
valid_df = pd.read_csv('/content/valid/_classes.csv')
test_df = pd.read_csv('/content/test/_classes.csv')

In [ ]:

train_df = pd.read_csv('/content/train/_classes.csv')
print(train_df.head())


                                          filename   army worm  \
0  118_jpg.rf.33f3543dc18f61284c8c547271b971ee.jpg           0   
1   48_jpg.rf.343afaa10fac0ce192d82bc237560b7d.jpg           0   
2  135_jpg.rf.343c0d9db6b5760008f26cd22a9686a1.jpg           0   
3   45_jpg.rf.353c0fa30afa6db870959deaa6cf7dea.jpg           0   
4  139_jpg.rf.361b0984c39459ea357f7617448c818c.jpg           0   

    legume blister beetle   red spider   rice gall midge   rice leaf roller  \
0                       0            0                 1                  0   
1                       0            0                 0                  0   
2                       0            1                 0                  0   
3                       0            0                 0                  1   
4                       0            0                 0                  0   

    rice leafhopper   rice water weevil   wheat phloeothrips  \
0                 0                   0                    0   


In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# Función para obtener etiquetas de un dataframe de one-hot encoding
def get_label_from_one_hot(dataframe):
    return dataframe.iloc[:, 2:].idxmax(axis=1)

# Cargar los datos de los archivos CSV
train_df = pd.read_csv('/content/train/_classes.csv')
valid_df = pd.read_csv('/content/valid/_classes.csv')
test_df = pd.read_csv('/content/test/_classes.csv')

# Convertir los dataframes de one-hot a una columna de etiquetas categóricas
train_df['label'] = get_label_from_one_hot(train_df)
valid_df['label'] = get_label_from_one_hot(valid_df)
test_df['label'] = get_label_from_one_hot(test_df)

# Crear la columna 'filename' que tenga el path completo de las imágenes
train_df['filename'] = train_df['filename'].apply(lambda x: f'/content/train/{x}')
valid_df['filename'] = valid_df['filename'].apply(lambda x: f'/content/valid/{x}')
test_df['filename'] = test_df['filename'].apply(lambda x: f'/content/test/{x}')

# Aumento de datos para el generador de entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# reescalado para los generadores de validación y prueba
test_datagen = ImageDataGenerator(rescale=1./255)

# generadores
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = test_datagen.flow_from_dataframe(
    dataframe=valid_df,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 696 validated image filenames belonging to 9 classes.
Found 199 validated image filenames belonging to 9 classes.
Found 100 validated image filenames belonging to 9 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


# Cargar MobileNetV2 preentrenado y excluir la capa superior
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Añadir nuevas capas personalizadas encima del modelo base
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(9, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



9406464/9406464 [==============================] - 0s 0us/step


In [1]:
#callbacks
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0)


In [ ]:
# Entrenar el modelo con los callbacks
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=25,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // valid_generator.batch_size,
    callbacks=[early_stopping, reduce_lr],
    verbose=2
)

# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy:.2f}, Test loss: {test_loss:.2f}')

Epoch 1/25
21/21 - 13s - loss: 1.8469 - accuracy: 0.4910 - val_loss: 0.5275 - val_accuracy: 0.8333 - lr: 0.0010 - 13s/epoch - 615ms/step
Epoch 2/25
21/21 - 4s - loss: 0.8745 - accuracy: 0.7304 - val_loss: 0.3898 - val_accuracy: 0.8854 - lr: 0.0010 - 4s/epoch - 193ms/step
Epoch 3/25
21/21 - 4s - loss: 0.6356 - accuracy: 0.7997 - val_loss: 0.3800 - val_accuracy: 0.8750 - lr: 0.0010 - 4s/epoch - 187ms/step
Epoch 4/25
21/21 - 5s - loss: 0.4815 - accuracy: 0.8438 - val_loss: 0.3323 - val_accuracy: 0.8906 - lr: 0.0010 - 5s/epoch - 235ms/step
Epoch 5/25
21/21 - 4s - loss: 0.4575 - accuracy: 0.8509 - val_loss: 0.3917 - val_accuracy: 0.8594 - lr: 0.0010 - 4s/epoch - 187ms/step
Epoch 6/25
21/21 - 4s - loss: 0.4619 - accuracy: 0.8614 - val_loss: 0.3203 - val_accuracy: 0.9010 - lr: 0.0010 - 4s/epoch - 188ms/step
Epoch 7/25
21/21 - 5s - loss: 0.3329 - accuracy: 0.8991 - val_loss: 0.3190 - val_accuracy: 0.8906 - lr: 0.0010 - 5s/epoch - 246ms/step
Epoch 8/25
21/21 - 4s - loss: 0.3395 - accuracy: 0.89

In [ ]:
#Guardar el modelo para descargarlo
model.save('mi_modelo2.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
import numpy as np


label_map = {v: k for k, v in train_generator.class_indices.items()}


# model = load_model('mi_modelo.h5')

def predict_image_class(image_path, model, label_map):
    # Cargar y preprocesar la imagen
    img = load_img(image_path, target_size=(150, 150))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Hacer la predicción
    predictions = model.predict(img_array)
    pred_index = np.argmax(predictions, axis=1)[0]
    pred_class = label_map[pred_index]

    return pred_class


test_image_path = '/content/test/113_jpg.rf.e7667502c16d6df0c7c57e0ef882f470.jpg'

# Usar la función para predecir la clase de la imagen
predicted_class = predict_image_class(test_image_path, model, label_map)
print(f'La imagen es: {predicted_class}')


1/1 [==============================] - 0s 23ms/step
La imagen es:  red spider
